In [50]:
import pandas as pd
import numpy as np
from random import sample
from pandas import *
import matplotlib.pyplot as plt

%matplotlib inline

In [51]:
# This section reads the csv files with the data from the SQL queries for the basic energy data and as-is and post wall data.  
# Note that for each file you will have to put in the  correct file pathname.
basic_data = pd.read_csv(r"C:\Users\dustin\Desktop\SEAR\calista_basic_energy_data.csv", index_col=None, header=0)
as_is_wall_data = pd.read_csv(r"C:\Users\dustin\Desktop\SEAR\calista_asis_wall_data.csv", index_col=None, header=0)
post_wall_data = pd.read_csv(r"C:\Users\dustin\Desktop\SEAR\calista_post_wall_data.csv", index_col=None, header=0)

In [52]:
# This section changes the name of the project ID column in the post wall data so it can be joined w/ basic energy data
new_columns = post_wall_data.columns.values;
new_columns[1] = 'Post_ProjectID'
post_wall_data.columns = new_columns

In [53]:
# This section changes the name of the project id column in the As-Is wall data so it can be joined with the basic energy data
new_asis_columns = as_is_wall_data.columns.values;
new_asis_columns[1] = 'Asis_ProjectID'
as_is_wall_data.columns = new_asis_columns

In [54]:
# This section appends the post wall data to the basic data
combined_data = merge(basic_data, post_wall_data, how='left', on="Post_ProjectID", sort=False, suffixes=('_bed', '_post'))

In [55]:
# Sorts the data so the largest wall component is first in the list, and then groups the data by project ID so there is only one wall
# component per ProjectID.
combined_data = combined_data.sort(['Post_ProjectID', 'GrossArea'], ascending=[True, False])
combined_data = combined_data.groupby('Post_ProjectID', as_index=False).first()
# combined_data.to_excel("Z_Test.xlsx", sheet_name="Post_Test")

In [56]:
# This section appends the as is wall data to the basic energy and post wall data. 
combined_data = merge(combined_data, as_is_wall_data, how='left', on="Asis_ProjectID", sort=False, suffixes=('_bed2', '_asis'))

In [57]:
# This section just lists the columns so they can be referenced
# list(combined_data.columns.values)

In [58]:
# Sorts the data so the largest wall component is first in the list, and then groups the data by project ID so there is only one wall
# component per As-Is ProjectID.
combined_data = combined_data.sort(['Asis_ProjectID', 'GrossArea_asis'], ascending=[True, False])
combined_data = combined_data.groupby('Asis_ProjectID', as_index=False).first()
# combined_data.to_excel("ZZ_Test.xlsx", sheet_name="Asis_Test")

In [59]:
# This section changes the name of the column because when something is labeled with a '.' like TypeID.2 it doesn't work.
change_columns = combined_data.columns.values;
change_columns[242] = 'asis_type_id'
combined_data.columns = change_columns

In [61]:
change_columns = combined_data.columns.values
change_columns[175] = 'post_type_id'
combined_data.columns = change_columns

In [62]:
# This section selects only the ratings which did not have sheathing in the As-Is file but do have sheathing in the Post file,
# indicating a sheathing retrofit was performed.
sheathing_retrofits = combined_data.query("asis_type_id != 14 and post_type_id == 14")

In [63]:
# This is the number of sheathing retrofits that have been performed in this region. 
sheathed_retro = len(sheathing_retrofits.index)
retrofits = len(combined_data.index)
print "Of approximately %d retrofits analyzed, %d added insulative sheathing." % (retrofits, sheathed_retro)

Of approximately 636 retrofits analyzed, 217 added insulative sheathing.


In [64]:
# Delete the pound sign below to print a test Excel file for checking if necessary
# sheathing_retrofits.to_excel("Sheating_retrofits_test.xlsx", sheet_name="Test")

In [65]:
# This section changes the column name to avoid the '.2' column naming problem
change_columns = sheathing_retrofits.columns.values;
change_columns[164] = 'sheathing_thickness'
sheathing_retrofits.columns = change_columns

In [66]:
# This section calculates the percentage of sheathing retrofits that added less than 2" of sheathing
little_sheathing = sheathing_retrofits.query("sheathing_thickness <= 2")
percent_less_two = float(len(little_sheathing.index)) / float(len(sheathing_retrofits.index)) * 100
print "%f percent of homes with sheathing retrofits added less than 2 in. of insulation." % percent_less_two

98.617512 percent of homes with sheathing retrofits added less than 2 in. of insulation.


In [67]:
# Just a check
# sheathing_retrofits['sheathing_thickness']

In [68]:
# This section calculates the ratio of sheathing R-value to total wall R-value for each row
sheathing_retrofits['sheath_rval_ratio'] = np.where(sheathing_retrofits['BattThickness_bed2'].isnull(), 
                                                    sheathing_retrofits['nominalRval_bed2'] * sheathing_retrofits['sheathing_thickness'] 
                                                    / sheathing_retrofits['Comp1R_bed2'], 
                                                    sheathing_retrofits['nominalRval_bed2'] / sheathing_retrofits['Comp1R_bed2'])
sheathing_retrofits['sheath_rval_ratio']
                                                    

1     0.257769
2     0.250345
3     0.250345
4     0.395089
5     0.250345
6     0.232046
8     0.464149
10    0.305960
12    0.284733
15    0.327031
16    0.257729
17    0.339276
18    0.339276
25    0.314342
26    0.339276
...
552    0.374861
553    0.504865
554    0.385410
556    0.369153
559    0.366140
563    0.374861
564    0.650638
565    0.302624
568    0.374861
569    0.394591
570    0.395089
571    0.366140
584    0.423982
607    0.423982
609    0.423982
Name: sheath_rval_ratio, Length: 217, dtype: float64

In [69]:
# sheathing_retrofits.to_excel("Sheating_retrofits_test3.xlsx", sheet_name="Test")

In [70]:
sheathing = sheathing_retrofits['sheath_rval_ratio'].median() * 100
cavity = 100 - sheathing
print "The median sheathing r-value ratio in homes with a sheathing retrofit is: %f percent sheathing insulation" % sheathing
print "and %f percent cavity insulation" % cavity

The median sheathing r-value ratio in homes with a sheathing retrofit is: 33.927614 percent sheathing insulation
and 66.072386 percent cavity insulation


In [71]:
add_sheath_add_vent = sheathing_retrofits.query("asis_VentType == 'None'")
add_sheath_add_vent = add_sheath_add_vent.query("post_VentType == 'HRV' or post_VentType == 'Mechancial_with_no_Heat_Recovery'")
add_sheath_add_vent_percent = float(len(add_sheath_add_vent.index)) / float(len(sheathing_retrofits.index)) * 100
add_sheath_no_vent_percent = 100 - add_sheath_add_vent_percent
print "The percentage of homes with a sheathing retrofit that also added mechanical ventilation is %f." % add_sheath_add_vent_percent

The percentage of homes with a sheathing retrofit that also added mechanical ventilation is 85.714286.


In [72]:
retrofit_no_vent = sheathing_retrofits.query("post_VentType == 'None'")
asis = retrofit_no_vent['asis_ACH50'].mean()
post = retrofit_no_vent['post_ACH50'].mean()
print "The other %f percent of retrofits without continuous mechanical ventilation became more air-tight: %f --> %f ACH50" % (add_sheath_no_vent_percent, asis, post)

The other 14.285714 percent of retrofits without continuous mechanical ventilation became more air-tight: 11.449657 --> 6.958659 ACH50


In [73]:
# list(combined_data.columns.values)

In [74]:
# Groups the data by the wwall's framing type and lumber dimensions
wall_analysis = combined_data.groupby(['WallType_bed2', 'Name_bed2']).count()
wall_analysis                               

Asis_ProjectID  Post_ProjectID  LocationID_bed  \
WallType_bed2       Name_bed2                                                   
Single_Stud         2 x 2                   1               1               1   
                    2 x 3                   1               1               1   
                    2 x 4                 179             179             179   
                    2 x 6                 133             133             133   
                    2 x 8                  13              13              13   
Strapped_Log        2 x 2                   1               1               1   
                    2 x 3                   1               1               1   
Strapped_Stud       2 x 3                   1               1               1   
Stressed_Skin_Panel 2 x 6                   3               3               3   
                    2 x 8                   1               1               1   

                               asis_address  asis_city  RatingType  YearBuilt  \
WallType_bed2       Name_bed2                                                   
Single_Stud         2 x 2                 1          1           1          1   
                    2 x 3                 1          1           1          1   
                    2 x 4               179        179         179        179   
                    2 x 6               133        133         133        133   
                    2 x 8                13         13          13         13   
Strapped_Log        2 x 2                 1          1           1          1   
                    2 x 3                 1          1           1          1   
Strapped_Stud       2 x 3                 1          1           1          1   
Stressed_Skin_Panel 2 x 6                 3          3           3          3   
                    2 x 8                 1          1           1          1   

                               asis_AirLoss  post_AirLoss  \
WallType_bed2       Name_bed2                               
Single_Stud         2 x 2                 1             1   
                    2 x 3                 1             1   
                    2 x 4               179           179   
                    2 x 6               133           133   
                    2 x 8                13            13   
Strapped_Log        2 x 2                 1             1   
                    2 x 3                 1             1   
Strapped_Stud       2 x 3                 1             1   
Stressed_Skin_Panel 2 x 6                 3             3   
                    2 x 8                 1             1   

                               asis_Applicances_loss          ...            \
WallType_bed2       Name_bed2                                 ...             
Single_Stud         2 x 2                          1          ...             
                    2 x 3                          1          ...             
                    2 x 4                        179          ...             
                    2 x 6                        133          ...             
                    2 x 8                         13          ...             
Strapped_Log        2 x 2                          1          ...             
                    2 x 3                          1          ...             
Strapped_Stud       2 x 3                          1          ...             
Stressed_Skin_Panel 2 x 6                          3          ...             
                    2 x 8                          1          ...             

                               Name.1_asis  BattThickness_asis  \
WallType_bed2       Name_bed2                                    
Single_Stud         2 x 2                0                   0   
                    2 x 3                0                   0   
                    2 x 4               36                   1   
                    2 x 6               63                   1   
       

In [75]:
# Calculates the percentage of each wall type 
wall_total = wall_analysis['Post_ProjectID'].sum()
wall_total = float(wall_total)
wall_analysis['Wall_Percentages'] = wall_analysis['Post_ProjectID'].map(lambda x: float(x) / wall_total * 100)
wall_analysis['Wall_Percentages']

WallType_bed2        Name_bed2
Single_Stud          2 x 2         0.299401
                     2 x 3         0.299401
                     2 x 4        53.592814
                     2 x 6        39.820359
                     2 x 8         3.892216
Strapped_Log         2 x 2         0.299401
                     2 x 3         0.299401
Strapped_Stud        2 x 3         0.299401
Stressed_Skin_Panel  2 x 6         0.898204
                     2 x 8         0.299401
Name: Wall_Percentages, dtype: float64